In [1]:
import numpy as np
import time
import pygame

def create_rot_matrix(angle: float):
    return np.array([[np.cos(angle), np.sin(angle)], 
                    [-np.sin(angle), np.cos(angle)]])
    
class Robot:
    length = None
    width = None
    corner_angle = None
    half_diag_length = None
    
    current_pos = None
    current_speed = None
    current_angular_velocity = None
    sensor_vals = [0, 0]
    
    angle = None
    direction_unit_vec = None
    
    corners = None
    corner_offsets = None

    def __init__(self, dimensions: tuple, start_pos: tuple, angle: float):
        self.width = dimensions[1]
        self.length = dimensions[0]
        self.current_speed = 0
        self.current_angular_velocity = 0
        self.corner_angle = np.arctan(self.width/self.length) #TThis is the angle b/w direction vector and centre->corner vector
        self.half_diag_length = np.linalg.norm([self.width/2, self.length/2]) #Half diagonal length is useful in calculations
        
        self.current_pos = np.array(start_pos, dtype='float64')
        self.angle = angle
        
        self.direction_unit_vec = create_rot_matrix(angle) @ np.array([1, 0])
        
        corner_0_offset = create_rot_matrix(angle + self.corner_angle) @ (self.half_diag_length * np.array([1, 0]))
        corner_1_offset = create_rot_matrix(angle - self.corner_angle) @ (self.half_diag_length * np.array([1, 0]))
        self.corner_offsets = np.array([corner_0_offset, corner_1_offset])

        self.corners = np.zeros((4, 2))
        self.corners[0] = self.current_pos + self.corner_offsets[0]
        self.corners[1] = self.current_pos + self.corner_offsets[1]
        self.corners[2] = self.current_pos - self.corner_offsets[0]
        self.corners[3] = self.current_pos - self.corner_offsets[1]

        self.wheel_pos = np.zeros((2, 2))
        self.wheel_pos[0] = (self.corners[0] + self.corners[3])/2
        self.wheel_pos[1] = (self.corners[1] + self.corners[2])/2
        
        ################################################
        #   Direction unit vector (init. dir = pi/2)   #
        #        D                                     #
        #        |                                     #
        #        |                                     #
        #   X,---|---,X Corner Offsets (O -> X)        #
        #    |\  |  /|                                 #
        #    | \ | / |                                 #
        #    |  \|/  |                                 #
        #    |   O   |  Length                         #
        #    |  pos  |                                 #
        #    |       |                 ^ Y             #
        #    |       |                 |               #
        #    `-------'                 |               #
        #      Width                   '-----> X       #
        ################################################


    # Helper functions
    # DO NOT TOUCH THESE
    def rotate(self, rot_angle: float):
        self.direction_unit_vec = create_rot_matrix(rot_angle) @ self.direction_unit_vec
        self.corner_offsets[0] = create_rot_matrix(rot_angle) @ self.corner_offsets[0]
        self.corner_offsets[1] = create_rot_matrix(rot_angle) @ self.corner_offsets[1]

        self.corners[0] = self.current_pos + self.corner_offsets[0]
        self.corners[1] = self.current_pos + self.corner_offsets[1]
        self.corners[2] = self.current_pos - self.corner_offsets[0]
        self.corners[3] = self.current_pos - self.corner_offsets[1]

        self.wheel_pos[0] = (self.corners[0] + self.corners[3])/2
        self.wheel_pos[1] = (self.corners[1] + self.corners[2])/2
        
    def move(self, displacement: float):
        self.current_pos += displacement
        for idx in range(4):
            self.corners[idx] += displacement

        self.wheel_pos[0] += displacement
        self.wheel_pos[1] += displacement

    # Access functions
    # You only ever have to use these functions to update the robots condition inside the emulator
    # DO NOT LET PARTICIPANTS DIRECTLY ACCESS THESE
    def update_angle(self, time_elapsed: float):
        self.rotate(self.current_angular_velocity * time_elapsed)

    def update_pos(self, time_elapsed: float):
        displacement = self.direction_unit_vec * self.current_speed * time_elapsed
        self.move(displacement)
        
    def set_speed(self, speed: float):
        self.current_speed = speed
        
    def set_ang_vel(self, ang_vel: float):
        self.current_angular_velocity = ang_vel

    def get_sensor_vals(self, screen: pygame.surface.Surface):
        '''
        reads the IR sensor and return a list
        SHOULD BE USED BEFORE UPDATING THE ROBOT ON SCREEN


        sensor_vals[0] -> left sensor
        sensor_vals[1] -> right sensor
        (assuming corners[0] is front left corner)

        '''
        colour1 = screen.get_at((int(self.corners[0][0]), int(self.corners[0][1])))
        colour1_gs = (colour1[0] + colour1[1] + colour1[2]) / 3
        colour2 = screen.get_at((int(self.corners[1][0]), int(self.corners[1][1])))
        colour2_gs = (colour2[0] + colour2[1] + colour2[2]) / 3
        if (colour1_gs < 150):
            self.sensor_vals[0] = 0
        else:
            self.sensor_vals[0] = 1
        if (colour2_gs < 150):
            self.sensor_vals[1] = 0
        else:
            self.sensor_vals[1] = 1

        #print(self.sensor_vals)

        return self.sensor_vals

pygame 2.5.2 (SDL 2.28.2, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# This class will be used by the participants to exchange data with the emulator
# The signal setter functions need to be changed to use the motor speed values
import pygame

MOTOR_MAX_SPEED = 60 #in rpm, which is 120pi rad/min, which is 2pi rad/s
WHEEL_RADIUS = 2.5 #in cm
class RobotData:
    '''
    - motor1speed: float IN
    - motor2speed: float IN
    - speed: float SIGNAL
    - ang_vel: float SIGNAL
    - sensor_values: [bool, bool] OUT
    '''
    def __init__(self, speed, ang_vel):
        self.current_speed = speed
        self.current_angular_velocity = ang_vel

    ############################################
    # DONT TOUCH THESE WHILE NOT DEBUGGING
    def set_speed(self, speed):
        self.speed = speed

    def set_ang_vel(self, ang_vel):
        self.ang_vel = ang_vel
    #############################################

    def get_speed(self):
        return self.current_speed
        
    def get_ang_vel(self):
        return self.current_angular_velocity

# Pygame implementation of environment

In [3]:
# Create map as string
# Can get map from files as string
# Made using asciiflow.comn

map = '''
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
   ────────────────┤          │                       │        ───┬──────────┬───────────────┤
                   │          │                       │           │          │               │
                   │          │                       │           │          │               │
                   │          │                       │           │                          │
                   │          │                       │           │                          ├───────────┐
                   │          ├──────────────         │           │                          │           │
                   │          │                       │           ├──────────────────────    │           │
                   │          │                       │           │                          │           │
   ────────────────┤          │                       │           │                          │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   ├──────────┴─────────────┬─────────┴─────────┬────────────────────────────┴───────────┴───┬────
                   │                        │                   │                                            │
                   │                        │                   │                                            │
                   │                        │                   │                      │                     │
                   │         │              │                   │                      │                     │
                   │         │              │                   ├────────────┬─────────┴──────               │
                   ├─────────┴────          │                   │            │                               │
                   │                        │                   │            │                               │
                   │                        ├────────────┐      │            │                               │
   ────────────────┤               │        │            │      │            │                               │
                   │               │        │            │      │            │
                   │               │        │            │      │            │
                   │               │        S            │      │            ├──────────────────────────────────
                   │               │                     │      │            │
                   │               │                     │      │            │
                   │               │                     │      │            │
                   │               │                     │      │            ├─────────┬──────────────┬───────────
                   │               │                     │      │            │         │              │
                   │               │                     │      │            │         │              │
                   ├───────────────┴─────────────────────┘      │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         └──────────────┴──────────G
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │

'''

# Convert string to rows of strings for easier iteration and position access
map_array = map.split('\n')

In [6]:
import pygame

SCREEN_WIDTH = 1800
SCREEN_HEIGHT = 1200
strip_width = 15 #1.5cm

ROBOT_WIDTH = 100 #10cm
ROBOT_LENGTH = 120 #12cm
##########################
# SCALE: 1cm = 10px
##########################

# Determine the start pos
start_pos = (0, 0)
for row in map_array:
    if 'S' in row:
        start_pos = np.array([row.index('S'), map_array.index(row)])
        print(start_pos)

# Temp variable Used once to center the robot onto the path at the start
path_offset = np.array([strip_width/2 - 2, strip_width/2])

# Create the robot object (Dimensions, start position, Direction facing)
my_rob = Robot((ROBOT_LENGTH, ROBOT_WIDTH), strip_width*start_pos + path_offset, np.pi/2)
robot_interface = RobotData(0, 0)

# Initialize the pygame objects and screen
pygame.init()
pygame.font.init()
my_font = pygame.font.SysFont('Roboto', 30)

screen = pygame.display.set_mode([SCREEN_WIDTH, SCREEN_HEIGHT])
running = True

rob_image = pygame.image.load('black.jpg')

rob_image_point = pygame.transform.scale(rob_image, (3, 3))
rob_image_wheel = pygame.transform.scale(rob_image, (5, 5))

# Time start and time end keep track of time elapsed in a loop
# Used for rendering time onto screen
# Also used for movement of robot
time_start = 0
time_end = 0
while running:
    time_start = time_end
    time_end = pygame.time.get_ticks()
    text_surface = my_font.render(str(pygame.time.get_ticks()), False, (0, 0, 0))

    my_rob.set_speed(robot_interface.get_speed())
    my_rob.set_ang_vel(robot_interface.get_ang_vel())
    my_rob.update_pos((time_end - time_start)/1000)
    my_rob.update_angle((time_end - time_start)/1000)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255)) #Fill background

    # Choosing j, i (position of block) and filling it in if it's a strip
    for j in range(len(map_array)):
        row = map_array[j]
        for i in range(len(row)):
            block_pos = (i*strip_width, j*strip_width) #Get block position
            if(row[i] in '─,│,┐,┘,└,┌,┬,┤,┴,├,┼'):
                pygame.draw.rect(screen, (100, 100, 100), block_pos + (strip_width, strip_width)) # Draw the path at block_pos
            elif row[i] == 'S':
                pygame.draw.rect(screen, (200, 200, 0), block_pos + (strip_width, strip_width)) # Draw start state
            elif row[i] == 'G':
                pygame.draw.rect(screen, (0, 200, 0), block_pos + (strip_width, strip_width)) # Draw goal state

    # TWO WHEELED ROBOT
    screen.blit(rob_image_point, my_rob.corners[0])
    screen.blit(rob_image_point, my_rob.corners[1])
    screen.blit(rob_image_point, my_rob.corners[2])
    screen.blit(rob_image_point, my_rob.corners[3])

    screen.blit(rob_image_wheel, my_rob.wheel_pos[0])
    screen.blit(rob_image_wheel, my_rob.wheel_pos[1])
    
    screen.blit(rob_image_point, my_rob.current_pos)
    screen.blit(rob_image_wheel, my_rob.current_pos + 30*my_rob.direction_unit_vec)
    screen.blit(text_surface, (500,0))
    pygame.display.flip()
    
pygame.quit()

[44 33]
